<a href="https://colab.research.google.com/github/Blueprint-GitHub/Study_Note/blob/main/what_is_polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pandas와 유사한 Polars 라이브러리에 대해서.**

**Polars의 특징과 장점**
1. 고성능: Rust 기반으로 고성능 처리가 가능.
2. 멀티스레딩: 자동적인 멀티스레딩을 지원하여 데이터 처리 속도를 향상.
3. 메모리 효율성: 데이터 저장 및 처리에 메모리 효율적인 방식을 사용.
4. 유연한 데이터 처리: SQL과 유사한 문법을 사용하여 복잡한 데이터 조작 수행 가능.
5. 대용량 데이터셋 처리: 큰 데이터셋에 대해 빠른 연산 제공.

In [ ]:
Polars는 인덱스가 없으므로 Pandas 처럼 train['data']나 train.loc[:, 'data']처럼 사용할 수 없고
train.select['data'] , train.filter(pl.col('data') < 5)처럼 사용해야 한다.
---------------------------------------------------------------------------------
### Polars의 주요 함수 및 메서드 ###

read_csv(): CSV 파일을 읽어 데이터 프레임을 생성.

data.write_csv("data.csv") : CSV 파일로 데이터 프레임 저장

drop(): 특정 열을 제거.
dropped_df = df.drop("unnecessary_column")
dropped_df = df.drop(["column1", "column2", "column3"])

filter(): 조건에 따라 데이터를 필터링.
filtered_df = df.filter(pl.col("age") > 30)

group_by() 및 agg(): 데이터를 그룹화하고 집계 함수를 적용.
grouped_df = df.groupby("department").agg([
    pl.col("salary").mean().alias("average_salary"),
    pl.col("salary").sum().alias("total_salary")])

sort(): 데이터를 특정 열에 따라 정렬.
sorted_df = df.sort("sort_column", reverse = False)
sorted_df = df.sort(["column1", "column2"])

join(): 다른 데이터 프레임과 결합.
joined_df = df1.join(df2, on="key_column", how="left")

select() : 데이터 프레임에서 특정 열만을 선택할 때 사용.
df.select(["column1", "column2"])

with_columns() : 기존 데이터 프레임에 새로운 열을 추가하거나, 기존 열을 변환한 결과를 추가할 때 사용.
df.with_columns(pl.col("existing_column").cast(pl.Float32))

col(): 특정 열을 선택할 때 사용.
df.select(pl.col("column_name"))

duration(): 날짜/시간 연산에 사용되며, 특정 기간을 나타냄.
df.with_columns((pl.col("datetime") + pl.duration(days=1)))

alias(): 열의 이름을 변경할 때 사용.
df.select(pl.col("old_name").alias("new_name"))

cast(): 열의 데이터 타입을 변경할 때 사용.
df.with_columns(pl.col("column_name").cast(pl.Float32))

In [ ]:
### 유용한 사용 예시 ###

# 데이터 변환
data.with_columns(pl.col(pl.Float64).cast(pl.Float32),)
-> float64 행을 모두 float32로 바꿈

# 체이닝 작업
result_df = (
    df.select(["column1", "column2", "datetime"])
      .filter(pl.col("column1") > 100)
      .with_columns(pl.col("datetime").dt.month().alias("month"))
)
->  1. column1, column2, datetime 열만 선택된 데이터 프레임.
    2. column1 값이 100보다 큰 행만 필터링됨.
    3. datetime 열에서 월을 추출하여 month라는 새로운 열이 추가됨.

# 조인
join(): 다른 데이터 프레임과 결합.
# 'key_column'을 기준으로 두 데이터 프레임을 조인
joined_df = df1.join(df2, on="key_column", how="left")
joined_df = df1.join(df2, on=["key1", "key2"], how="inner")
joined_df = df1.join(df2, on="key_column", how="left", suffix="_df2")
->  1. 조인 유형은 left, right, inner, outer 중에 하나를 선택한다.
    2. 다중 조인을 사용할때는 키(on)를 리스트로 전달한다.
    3. 조인 시 충돌하는 열 이름을 처리하기 위해 suffix 매개변수를 사용할 수 있다.

# 시간대 처리
df_time = df.with_columns(
    pl.col("datetime").dt.convert_time_zone("UTC", "Europe/Berlin").alias("berlin_time")
)
->  1. datetime 열의 시간대를 UTC에서 "Europe/Berlin"으로 변환함.
    2. 새로운 열 berlin_time이 데이터 프레임에 추가됨.

# 시간대 기반 조인
joined_df = df.join(
    other_df,
    left_on=pl.col("datetime"),
    right_on=pl.col("datetime").dt.truncate("hour")
)
->  1. 'datetime' 열을 기준으로 두 데이터 프레임을 조인
       (여기서는 'datetime' 열을 시간 단위로 절삭(truncate)하여 조인)
    2. other_df의 datetime 열은 시간 단위로 절삭되어, df의 datetime과 일치하는
       가장 가까운 시간에 해당하는 행과 조인됨

시간대가 서로 다른 데이터를 합칠때는 다음과 같이 한 쪽의 시간대를 filter로 조정한 뒤 합칠 수 있다.
forecast_filtered = forecast.filter(pl.col('datetime') <= "2021-05-30 10:00")
df_merged = forecast_filtered.join(historical, on='datetime', how='left')


# 열 결합
combined_df = df.with_columns(
    (pl.col("first_name") + " " + pl.col("last_name")).alias("full_name")
)
-> 'first_name'과 'last_name' 열을 결합하여 새로운 'full_name' 열을 생성

# 윈도우 함수
rolling_df = df.with_columns(
    pl.col("value").rolling_mean(window_size=3).alias("rolling_mean")
)
-> 'value' 열에 대해 3개 행의 윈도우 크기로 이동 평균을 계산
    value 열에 대한 이동 평균을 계산한 새로운 열 rolling_mean을 포함한 데이터 프레임을 반환함.
    각 행의 rolling_mean 값은 해당 행을 포함하여 이전 두 행의 value 값의 평균을 나타냄.

# 피벗 테이블
pivot_df = df.pivot(
    index="index_column",
    columns="columns_column",
    values="values_column"
).fill_none(0)
->  'index_column'을 인덱스로, 'columns_column'을 열로 사용하여 피벗 테이블 생성
    'values_column'을 값으로 사용(값이 없으면 0으로 채워짐)